In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import datetime as dt
import re
from dateutil.relativedelta import relativedelta
from string import ascii_uppercase

In [2]:
# Loading the raw data. Specified a column name as numbers in object to see easily.
df=pd.read_excel("/Users/suinkim/Downloads/TEST.xlsx",usecols=list(range(1,25,+1)),header=None)

In [28]:
df3 = df.fillna("")
df3

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,YEAR 1,,,FULL YEAR,,,,,...,,,,,YEAR 3,,,FULL YEAR,,
4,,,Fincial Metric,,,Weight,Times,outcome,,,...,Times,outcome,,,Fincial Metric,,,Weight,Times,outcome
5,,,Fincial Metric 1,,,100,1,Y,,,...,10,Y,,,Fincial Metric 1,,,21213,50,Y
6,,,Fincial Metric 2,,,200,2,N,,,...,20,N,,,Fincial Metric 2,,,55567,60,N
7,,,Fincial Metric 3,,,300,3,Y,,,...,30,Y,,,Fincial Metric 3,,,89890,70,Y
8,,,Fincial Metric 4,,,400,4,N,,,...,40,N,,,Fincial Metric 4,,,40524,80,N
9,,,,,,,,,,,...,,,,,,,,,,


In [5]:
df2=df.rename(columns={3:'Comment',19:'Score'})

In [6]:
# Filter only relevent data
df2=df2[["Comment","Score"]]

In [7]:
df2=df2[11:]

In [8]:
# Getting raw data as a set of list since columns and raw are unstructured 
lst = df2["Comment"].tolist() + df2["Score"].tolist()

In [9]:
# Filtering NAN out
cleanedlst=[x for x in lst if str(x) != 'nan']

In [10]:
# Setting columns
col_names=cleanedlst[::2]

In [11]:
# Setting values
val = cleanedlst[1::2]

In [12]:
# Expressing in DataFrame
data={
    'Comment':val[col_names.index('Comment')],
    'Result':val[col_names.index('Result')],
    "Score":val[col_names.index("Score")]
}

In [13]:
df2=pd.DataFrame(data,index=[0])

In [14]:
# Arrange an ID to join with Years table. 
df2["id"]='1'

In [15]:
df2

,Comment,Result,Score,id
0,Please review something,Passed,A-2,1


In [16]:
# Full Year 1
#df = df.fillna("")
df.dropna(how='all')
df_year1=df.loc[4:8,3:8]
df_year1=df_year1[[3,6,7,8]]
df_year1.columns=df_year1.iloc[0]
df_year1["Year"]='1'
df_year1=df_year1[["Year","Fincial Metric","Weight","Times","outcome"]]
df_year1=df_year1[1:]

In [17]:
# Full Year 2
df_year2=df.loc[4:8,10:16]
df_year2=df_year2[[11,14,15,16]]
df_year2.columns=df_year2.iloc[0]
df_year2["Year"]='2'
df_year2=df_year2[["Year","Fincial Metric","Weight","Times","outcome"]]
df_year2=df_year2[1:]

In [18]:
# Full Year 3
df_year3=df.loc[4:8,18:24]
df_year3=df_year3[[19,22,23,24]]
df_year3.columns=df_year3.iloc[0]
df_year3["Year"]='3'
df_year3=df_year3[["Year","Fincial Metric","Weight","Times","outcome"]]
df_year3=df_year3[1:]

In [19]:
# Concat year1, year2 and year3. Can express year1,year2,year3 seperately as your favor.
df_final=pd.concat([df_year1,df_year2,df_year3],ignore_index=True)

In [20]:
# Arrange an ID to join with other table. This can be also a sheet number.
df_final["id"]='1'

In [21]:
df_final

4,Year,Fincial Metric,Weight,Times,outcome,id
0,1,Fincial Metric 1,100,1,Y,1
1,1,Fincial Metric 2,200,2,N,1
2,1,Fincial Metric 3,300,3,Y,1
3,1,Fincial Metric 4,400,4,N,1
4,2,Fincial Metric 1,6565,10,Y,1
5,2,Fincial Metric 2,3900,20,N,1
6,2,Fincial Metric 3,5621,30,Y,1
7,2,Fincial Metric 4,6049,40,N,1
8,3,Fincial Metric 1,21213,50,Y,1
9,3,Fincial Metric 2,55567,60,N,1


In [22]:
# Create a new table by merging a common table, ID.
df_final=df_final.merge(df2,left_on="id", right_on='id')

In [23]:
# Setting Primary Key
df_final["index"]=range(1,len(df_final)+1)

In [24]:
df_final

,Year,Fincial Metric,Weight,Times,outcome,id,Comment,Result,Score,index
0,1,Fincial Metric 1,100,1,Y,1,Please review something,Passed,A-2,1
1,1,Fincial Metric 2,200,2,N,1,Please review something,Passed,A-2,2
2,1,Fincial Metric 3,300,3,Y,1,Please review something,Passed,A-2,3
3,1,Fincial Metric 4,400,4,N,1,Please review something,Passed,A-2,4
4,2,Fincial Metric 1,6565,10,Y,1,Please review something,Passed,A-2,5
5,2,Fincial Metric 2,3900,20,N,1,Please review something,Passed,A-2,6
6,2,Fincial Metric 3,5621,30,Y,1,Please review something,Passed,A-2,7
7,2,Fincial Metric 4,6049,40,N,1,Please review something,Passed,A-2,8
8,3,Fincial Metric 1,21213,50,Y,1,Please review something,Passed,A-2,9
9,3,Fincial Metric 2,55567,60,N,1,Please review something,Passed,A-2,10


In [ ]:
# Connect SQL 
server = '10.1.3.25' 
database = '**' 
username = '**' 
password = '**' 
connection_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
print("connected")

In [ ]:
# Store into Database
df_final.to_sql('fact.ETL', engine, schema = "dbo", if_exists='append', index=False, chunksize=10000)
print("\n Successfully Transported")